Manual training data collection for Crozier videos

In [1]:
import cv2
import os
from tqdm import tqdm

In [11]:
def extract_and_save_frames(video_path, video_name, time_ranges, output_folder, n=5):
    # Open the video file
    cam = cv2.VideoCapture(video_path)

    # Get video fps and total number of frames
    fps = cam.get(cv2.CAP_PROP_FPS)
    if fps == 0:
        print("Error: Could not retrieve video FPS. Terminating.")
        return
    print(fps)
    total_frames = int(cam.get(cv2.CAP_PROP_FRAME_COUNT))

    # Convert time ranges into frame ranges
    frame_ranges = []
    for idx, time_range in enumerate(time_ranges):
        start, end = time_range
        start_frame = int(start * fps)
        end_frame = int(end * fps)
        frame_ranges.append((start_frame, end_frame))

    # Create output folder for other frames (not indicated in time_range)
    outside_folder = os.path.join(output_folder, "other_frames")
    os.makedirs(outside_folder, exist_ok=True)

    # Create folders for each indicated time range
    range_folders = []
    for idx in range(len(frame_ranges)):
        range_folder = os.path.join(output_folder, f"deficiency_{idx + 1}")
        os.makedirs(range_folder, exist_ok=True)
        range_folders.append(range_folder)

    # Iterate through all frames in the video, skipping by 'n' steps
    for frame_number in tqdm(range(0, total_frames, n)):
        # Check if the frame falls into any of the time ranges
        saved_in_range = False
        for idx, (start_frame, end_frame) in enumerate(frame_ranges):
            if start_frame <= frame_number <= end_frame:
                # Frame belongs to this range
                cam.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
                success, frame = cam.read()
                if success:
                    output_path = os.path.join(range_folders[idx], f"{video_name}_frame_{frame_number}.jpg")
                    cv2.imwrite(output_path, frame)
                saved_in_range = True
                break

        # If the frame doesn't belong to any range, save it in the outside folder
        if not saved_in_range:
            cam.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
            success, frame = cam.read()
            if success:
                output_path = os.path.join(outside_folder, f"{video_name}_frame_{frame_number}.jpg")
                cv2.imwrite(output_path, frame)

    # Release the video capture object
    cam.release()

In [22]:
video_path = "/content/SAP 34498 SAP 34499_SAP 34499_202311021430.mp4"
video_name = "SAP_34498_SAP_34499_SAP_34499"
output_folder = "/content"
time_ranges = []  # Timestamps in seconds

#extract_and_save_frames_from_timestamps(video_path, time_ranges, output_folder)
extract_and_save_frames(video_path, video_name, time_ranges, output_folder, n=25)

30.0


100%|██████████| 1260/1260 [03:37<00:00,  5.80it/s]


In [23]:
!zip -r /content/other_frames.zip /content/other_frames

  adding: content/other_frames/ (stored 0%)
  adding: content/other_frames/SAP_34498_SAP_34499_SAP_34499_frame_26750.jpg (deflated 0%)
  adding: content/other_frames/SAP_34498_SAP_34499_SAP_34499_frame_14925.jpg (deflated 1%)
  adding: content/other_frames/SAP_34498_SAP_34499_SAP_34499_frame_19700.jpg (deflated 1%)
  adding: content/other_frames/SAP_34498_SAP_34499_SAP_34499_frame_31325.jpg (deflated 0%)
  adding: content/other_frames/SAP_34498_SAP_34499_SAP_34499_frame_20425.jpg (deflated 0%)
  adding: content/other_frames/SAP_34498_SAP_34499_SAP_34499_frame_4900.jpg (deflated 1%)
  adding: content/other_frames/SAP_34498_SAP_34499_SAP_34499_frame_6200.jpg (deflated 1%)
  adding: content/other_frames/SAP_34498_SAP_34499_SAP_34499_frame_400.jpg (deflated 1%)
  adding: content/other_frames/SAP_34498_SAP_34499_SAP_34499_frame_29225.jpg (deflated 0%)
  adding: content/other_frames/SAP_34498_SAP_34499_SAP_34499_frame_23325.jpg (deflated 0%)
  adding: content/other_frames/SAP_34498_SAP_34499

In [24]:
import shutil

shutil.rmtree('/content/other_frames')